In [6]:
import torch
import pandas as pd
import os,natsort
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tnrange, tqdm
from modelTraining.modelDefinition import get_instance_segmentation_model
from DatasetUtils.FimgHandling import normalizeImg,cutTrayInIndividualImages, prepareForModel
from DatasetUtils.ConvenientFunctions import get_average_and_store, create_time_series

import matplotlib
matplotlib.use("Agg")
sns.reset_defaults()
sns.set_style("whitegrid")

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model =get_instance_segmentation_model(2)
stateDict = torch.load("Models/Leaf_Segmentation_MaskedRCNN_7_7_2022_8h.h5",map_location=device)
model.load_state_dict(stateDict)
model.to(device)
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

# Images ordering
As the dataset requires a previous work on ordering due to export labeling issues, here we provide the solution

In [10]:
folder_path1 = "Datasets/Tracking_validation_dataset/ParametersImages_8_DAS_17"
folder_path2 = "Datasets/Tracking_validation_dataset/ParametersImages_18_DAS_26"
folder_path3 = "Datasets/Tracking_validation_dataset/ParametersImages"
tray_ID = "002"
files1 = os.listdir(folder_path1)
files1 = [e for e in files1 if e.endswith("Fmp.fimg")]
files1 = natsort.natsorted([folder_path1+"/"+e for e in files1 if tray_ID in e])
files2 = os.listdir(folder_path2)
files2 = [e for e in files2 if e.endswith("Fmp.fimg")]
files2 = natsort.natsorted([folder_path2+"/"+e for e in files2 if tray_ID in e])
files12 =files1 + files2
files3 = os.listdir(folder_path3)
files3 = [e for e in files3 if (e.endswith("Fm.fimg") and not (e.endswith("Fv_Fm.fimg")))]
files3 = natsort.natsorted([folder_path3+"/"+e for e in files3 if tray_ID in e])
files = []
for i in range(len(files3)):
    files += [files3[0]]
    files +=files12[0:2]
    files3.pop(0)
    files12.pop(0)
    files12.pop(0)
tray = []
normalized_tray = []
for i in tnrange(len(files)):
    fimg_path = files[i]
    this_tray = cutTrayInIndividualImages(fimg_path,normalized=False)
    images = []
    for e in this_tray:
        img = e.copy()
        img[img < 0]= 0
        images.append(img)
    tray.append(images)
    this_normalized_tray = [normalizeImg(e,torch_format=True) for e in images]
    this_normalized_tray = prepareForModel(this_normalized_tray)
    normalized_tray.append(this_normalized_tray)
folder_path1 = "Datasets/Tracking_validation_dataset/ParametersImages_8_DAS_17"
folder_path2 = "Datasets/Tracking_validation_dataset/ParametersImages_18_DAS_26"
folder_path3 = "Datasets/Tracking_validation_dataset/ParametersImages"
files1 = os.listdir(folder_path1)
files1 = [e for e in files1 if e.endswith("Fp.fimg")]
files1 = natsort.natsorted([folder_path1+"/"+e for e in files1 if tray_ID in e])
files2 = os.listdir(folder_path2)
files2 = [e for e in files2 if e.endswith("Fp.fimg")]
files2 = natsort.natsorted([folder_path2+"/"+e for e in files2 if tray_ID in e])
files12 =files1 + files2
files3 = os.listdir(folder_path3)
files3 = [e for e in files3 if (e.endswith("Fo.fimg") and not (e.endswith("Fv_Fm.fimg")))]
files3 = natsort.natsorted([folder_path3+"/"+e for e in files3 if tray_ID in e])
files = []
for i in range(len(files3)):
    files += [files3[0]]
    files +=files12[0:2]
    files3.pop(0)
    files12.pop(0)
    files12.pop(0)

tray_fo = []
for i in tnrange(len(files)):
    fimg_path = files[i]
    this_tray_fo = cutTrayInIndividualImages(fimg_path, normalized=False)
    images = []
    for e in this_tray_fo:
        img = e.copy()
        img[img < 0] = 0
        images.append(img)
    tray_fo.append(images)

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

# Model usage
Eventhough this cell is does not require GPU to run, it is highly encouraged.

In [14]:
if device.type == "cuda":
    results = []
    for i in tqdm(normalized_tray):
        i = i.cuda()
        this_result = model(i)
        for idx,result in enumerate(this_result):
            for k,v in result.items():
                result[k] = v.detach().cpu()
            this_result[idx] = result
        results.append(this_result)
        del result
        del v
        del this_result
        del i
        torch.cuda.empty_cache()
else:
    results = []
    for i in tqdm(normalized_tray):
        this_result=model(i)
        for idx,result in enumerate(this_result):
            for k,v in result.items():
                result[k] = v.detach()
            this_result[idx] = result
        results.append(this_result)
        del this_result
        del i
        del v
        del result

  0%|          | 0/57 [00:00<?, ?it/s]

# Running the association process and information store

In the following cell we perform the tracking as described in the manuscript. We also perform the PSII, Fmp, and Fp analysis and estimation, all wrapped in a function

In [15]:
trays_order =["A1","B1","C1","A2","B2","C2","A3","B3","C3"]
tray_number = "002"
tray_name = "example_tray"
for i in tnrange(9):

    name = trays_order[i]
    time_series, series,images = create_time_series(tray,results,index=i)


    get_average_and_store(path="{}_{}".format(tray_name,tray_number),
                          time_series=time_series,
                          series=series,
                          images=images,
                          name=name,
                          line_plot=True,
                          gif=True,
                          save_masks=True,
                          save_fcam_values=True,
                          type="Fmp")
    images_fo = []
    for e in tray_fo:
        img = e[i].copy()
        img[img < 0]= 0
        images_fo.append(img)
    get_average_and_store(path="{}_{}".format(tray_name,tray_number),
                          time_series=time_series,
                          series=series,
                          images=images_fo,
                          name=name,
                          line_plot=True,
                          gif=False,
                          save_masks=False,
                          save_fcam_values=True,
                          type="Fo")
    df1 = pd.read_csv("{}_{}/{}/{}_Fmp_Average_Flcam_values_per_leaf.tsv".format(tray_name,tray_number,name,name),
                      sep="\t",
                      index_col=0)
    df2 = pd.read_csv("{}_{}/{}/{}_Fo_Average_Flcam_values_per_leaf.tsv".format(tray_name,tray_number,name,name),
                      sep="\t",
                      index_col=0)
    val = (df1.values - df2.values)
    val[val < 0] = 0
    val = val/df1.values
    total_average = pd.DataFrame(val)
    total_average.to_csv("{}_{}/{}/{}_phiPSII_Average_Flcam_values_per_leaf.tsv".format(tray_name,tray_number,name,name),
                             sep="\t")
    plt.figure(figsize=(12,8))
    sns.lineplot(data=total_average)
    plt.xticks(total_average.index.tolist())
    plt.legend()
    plt.savefig(os.path.join("{}_{}/{}".format(tray_name,tray_number,name,name)
                             ,"{}_phiPSII_graph.png".format(name)),dpi=300)

  0%|          | 0/9 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '002_A1/A1\\A1_phiPSII_graph.png'